Turning in half done. I left for my technical interview right when Alexis started talking about the week's lesson and came back just in time to be thoroughly lost. I was really banking on being able to watch the video to try get this done. I was able to do the first one based on the code from Alexis, but I really need to see everything in action to get the second one. Hopefully I'll be able to come back and finish it once the video is posted. 

# Use Splinter and/or Beautiful Soup to scrape https://www.dndbeyond.com/classes, and pull out all class names. Store these in a dataframe.


In [17]:
import pandas as pd


In [18]:
from bs4 import BeautifulSoup as bs
import requests
url = 'http://bookstoscrape.com'

In [19]:
response = requests.get(url)
response

<Response [200]>

In [20]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager


In [27]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'http://dndbeyond.com/classes'
browser.visit(url) 



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\weird\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [29]:
#I've played Rogue and Fighter. I think I liked the rogue the best.

soup = bs(browser.html, 'html.parser')
all_class = soup.find_all('h3', class_='listing-card__title')
dnd_list = []
for dndclass in all_class:
    dnd_dict = {"class":dndclass.text.strip(' ').strip('\n')}
    dnd_list.append(dnd_dict)
dnd_df =  pd.DataFrame(dnd_list)
dnd_df

,class
0,Barbarian
1,Bard
2,Cleric
3,Druid
4,Fighter
5,Monk
6,Paladin
7,Ranger
8,Rogue
9,Sorcerer


# Now, alter 1 to click on each class name to go to the more detailed page, and pull out the first paragraph about each class. Store these in a dataframe. Your dataframe should have two columns

In [ ]:
url = 'http://dndbeyond.com/classes'
browser.visit(url)
soup = bs(browser.html, 'html.parser')
all_class = soup.find_all('h3', class_='listing-card__title')
dnd_list = []